# NOTE

I said I used a custom webscraping script in the video, but I actually changed it back after recording last minute because I found a better way to scrape using the tweepy api


Importing Modules

In [109]:
import os
import tensorflow as tf
import tweepy

In [110]:
!pip install nltk

In [111]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
import re
from nltk.tokenize import word_tokenize
import gensim
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam

In [112]:
plt.style.use('ggplot')
nltk.download("stopwords")
nltk.download('punkt')
stop=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [113]:
!pip install pyspellchecker

In [114]:
from spellchecker import SpellChecker

# Setting up Model
I trained the model on an external website called Kaggle.com where I had longer access to a GPU

In [115]:
model_path = "drive/My Drive/NLP Congress Project/le_model"

In [116]:
model = tf.keras.models.load_model(model_path)

In [117]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 100)           2034300   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 50, 100)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 2,076,605
Trainable params: 42,305
Non-trainable params: 2,034,300
_________________________________________________________________


In [118]:
THRESHOLD = 0.85

# Data Processing Set Up for AI

In [119]:
spell = SpellChecker()

def pred(tweet):
  
  arr = []
  for t in tweet:
    arr.append([np.nan, np.nan, t])

  df = pd.DataFrame(np.array(arr),columns=["keyword", "location", "text"])


  def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

  def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

  def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

  def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)
  def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

  df['text']=df['text'].apply(lambda x : remove_URL(x))
  df['text']=df['text'].apply(lambda x : remove_html(x))
  df['text']=df['text'].apply(lambda x : remove_emoji(x))
  df['text']=df['text'].apply(lambda x : remove_punct(x))
  
  def create_corpus(df):
    corpus=[]
    for tweet in tqdm(df['text']):
        words=[word.lower() for word in word_tokenize(tweet) if((word.isalpha()==1) & (word not in stop))]
        corpus.append(words)
    return corpus

  corpus = create_corpus(df)

  embedding_dict={}
  with open('drive/My Drive/NLP Congress Project/glove/glove.6B.100d.txt','r') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        embedding_dict[word]=vectors
  f.close()

  MAX_LEN=50
  tokenizer_obj=Tokenizer()
  tokenizer_obj.fit_on_texts(corpus)
  sequences=tokenizer_obj.texts_to_sequences(corpus)

  tweet_pad=pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')

  word_index=tokenizer_obj.word_index

  num_words=len(word_index)+1
  embedding_matrix=np.zeros((num_words,100))

  for word,i in tqdm(word_index.items()):
    if i > num_words:
        continue
    
    emb_vec=embedding_dict.get(word)
    if emb_vec is not None:
        embedding_matrix[i]=emb_vec

  pred =  model.predict(tweet_pad[:df.shape[0]])

  arr = []


  for p in pred:
    if p >= THRESHOLD:
      arr.append(True)
    else:
      arr.append(False)
  return arr, df["text"].tolist()

# Tweepy Scraper

In [120]:
!pip install twilio

In [121]:
import os
from twilio.rest import Client

account_sid = 'ACddf1bcdc71a200ecd2b20cae7b489186'
auth_token = 'f972a857f5fcc6596f833d403971a42b'

client = Client(account_sid, auth_token)

def send_sms(from_, to, body):
    return client.messages.create(from_=from_, to=to, body=body)

In [122]:
from datetime import datetime
import time
import requests
from bs4 import BeautifulSoup as soup


global WATCH_USER
global TIME_SLEEP
jobs = []

TIME_SLEEP = 300

In [123]:
from datetime import date

In [124]:
# Tweepy is a Python module for interacting with the Twitter API
import tweepy, time, sys
# Enter the corresponding information from your Twitter application:
CONSUMER_KEY = 'sT8tMpB8lrvMQwk9XUmYVAr1L'# Keep the quotes, replace this with you
CONSUMER_SECRET = 'tzf6Sd6GKm5s1TbyhxyARcvIi1p3PJt0EMduhetJIIk7nts6jX' # Keep the
ACCESS_TOKEN = '837839702535385089-NJa7IgJ3h0XvuQaVRdYo8KCIcJCmUsE' # Keep the quo
ACCESS_SECRET = '2BXFUabtnNzH5OaSUZgEY8t6lRpFeGGDfMN3KU4hEQyAg' # Keep the quotes,
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tweepy.API(auth) # This creates the API object

# Setting up the Front End
Now, the back end is more or less complete. All that is left is to set up the actual web page with HTML and Flask

In [125]:
!pip install flask-ngrok
!pip install flask
!pip install flask-socketio

In [126]:
def mergeSort(arr): 
    if len(arr) >1: 
        mid = len(arr)//2 
        L = arr[:mid]
        R = arr[mid:]
  
        mergeSort(L)
        mergeSort(R) 
  
        i = j = k = 0

        while i < len(L) and j < len(R): 
            if L[i][1] < R[j][1]: 
                arr[k] = L[i] 
                i+= 1
            else: 
                arr[k] = R[j] 
                j+= 1
            k+= 1

        while i < len(L): 
            arr[k] = L[i] 
            i+= 1
            k+= 1
          
        while j < len(R): 
            arr[k] = R[j] 
            j+= 1
            k+= 1

In [127]:
def word_anal(arr):
  words = []
  output = []
  for i in arr:
    item = i.split(" ")
    for word in item:
      if word not in words:
        words.append(word)
        output.append([word, 0])
      else:
        output[words.index(word)][1] += 1
  mergeSort(output)
  result = []
  for i in output[-5:]:
    if str(i[0]) != "":
      result.append(str(i[0]))
  return result

In [128]:
def set_latest(arr):
  id, time, tweet = arr[0], arr[1], arr[2]
  return """
  <div class="w3-third w3-section">
          <span class="w3-xlarge">""" + str(id) + """ </span><br>
      """ + str(time) + """
        </div>
        <div class="w3-twothird w3-section">
          <span class="w3-xlarge">Tweet:</span><br>
          """+ str(tweet) + """
        </div>
    """

def set_recent_ten(tweets, filtered):
  output = ""
  if len(tweets) < 10:
    for t in tweets:
      id, time, tweet = t[0], t[1], t[2]
      output += """
      <p><span class="w3-large w3-margin-right">""" + str(id) + """</span>""" + str(time) + """</p>
      <p>""" + str(tweet) + """</p><br>"""
  else:
    for t in tweets[-10:]:
      id, time, tweet = t[0], t[1], t[2]
      output += """
      <p><span class="w3-large w3-margin-right">""" + str(id) + """</span>""" + str(time) + """</p>
      <p>""" + str(tweet) + """</p><br>"""
  return output

def set_words(tweets):
  words = word_anal(tweets)
  words = ["Schaumburg", "Woodfield", "Sears"]
  output = ""
  for i in range(len(words)):
    output += """<p><span class="w3-large w3-margin-right">""" + str(i+1) + """.</span>  """ + str(words[i]) + """</p>""" 
  return output

In [129]:
def get_page(tweets, city, state, filtered):
  return """
<!DOCTYPE html>
<html>
<title>""" + city + ", " + state + """</title>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1">
<link rel="stylesheet" href="https://www.w3schools.com/w3css/4/w3.css">
<link rel="stylesheet" href="https://fonts.googleapis.com/css?family=Montserrat">
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
<style>
body, h1,h2,h3,h4,h5,h6 {font-family: "Montserrat", sans-serif}
.w3-row-padding img {margin-bottom: 12px}
/* Set the width of the sidebar to 120px */
.w3-sidebar {width: 120px;background: #222;}
/* Add a left margin to the "page content" that matches the width of the sidebar (120px) */
#main {margin-left: 120px}
/* Remove margins from "page content" on small screens */
@media only screen and (max-width: 600px) {#main {margin-left: 0}}
</style>
<body class="w3-black">

<!-- Navbar on small screens (Hidden on medium and large screens) -->
<div class="w3-top w3-hide-large w3-hide-medium" id="myNavbar">
  <div class="w3-bar w3-black w3-opacity w3-hover-opacity-off w3-center w3-small">
    <a href="#" class="w3-bar-item w3-button" style="width:25% !important">HOME</a>
    <a href="#about" class="w3-bar-item w3-button" style="width:25% !important">ABOUT</a>
  </div>
</div>

<!-- Page Content -->
<div class="w3-padding-large" id="main">
  <!-- Header/Home -->
  <header class="w3-container w3-padding-32 w3-center w3-black" id="home">
    <h1 class="w3-jumbo"><span class="w3-hide-small">TweetifAI</span></h1>
    <img src="https://ak.picdn.net/shutterstock/videos/29840074/thumb/1.jpg" alt="boy" class="w3-image" width="600" height="600">
  </header>

  <!-- About Section -->
  <div class="w3-content w3-justify w3-text-grey w3-padding-64" id="about">
	<h3 class="w3-padding-16 w3-text-light-grey">Latest Tweet in """ + city + ", " + state +  """ </h3>
	  <div class="w3-row w3-center w3-padding-16 w3-section w3-light-grey">
""" + set_latest(tweets[-1]) + """
    </div>
	<!-- Testimonials -->
      <h3 class="w3-padding-24 w3-text-light-grey">Common Disaster Terms in """ + city + ", " + state + """ Right Now</h3>""" + set_words(filtered) + """
        <!-- Testimonials -->
    <h3 class="w3-padding-24 w3-text-light-grey">Recent Tweets in """ + city + ", " + state +  """</h3>  """ + set_recent_ten(tweets, filtered) + """
  <!-- End About Section -->
  </div>

  <!-- Contact Section -->
  <div class="w3-padding-64 w3-content w3-text-grey" id="contact">
    <p>Submit Phone Number:</p>

    <form action="/action_page.php" target="_blank">
      <p><input class="w3-input w3-padding-16" type="text" placeholder="Phone Number" required name="Number"></p>
      <p>
        <button class="w3-button w3-light-grey w3-padding-large" type="submit">
          <i class="fa fa-paper-plane"></i> SUBMIT NUMBER
        </button>
      </p>
    </form>
  <!-- End Contact Section -->
  </div>
 
    <!-- Footer -->
  <footer class="w3-content w3-padding-64 w3-text-grey w3-xlarge">
    <p class="w3-medium">Powered by <a href="https://www.w3schools.com/w3css/default.asp" target="_blank" class="w3-hover-text-green">w3.css</a></p>
  <!-- End footer -->
  </footer>

<!-- END PAGE CONTENT -->
</div>

</body>
</html>"""

In [130]:
def get_blank(city, state):
  query = city + ", " + state
  return """
<!DOCTYPE html>
<html>
<title>""" + query + """</title>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1">

<link rel="stylesheet" href="https://www.w3schools.com/w3css/4/w3.css">
<link rel="stylesheet" href="https://fonts.googleapis.com/css?family=Montserrat">
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
<style>
body, h1,h2,h3,h4,h5,h6 {font-family: "Montserrat", sans-serif}
.w3-row-padding img {margin-bottom: 12px}
/* Set the width of the sidebar to 120px */
.w3-sidebar {width: 120px;background: #222;}
/* Add a left margin to the "page content" that matches the width of the sidebar (120px) */
#main {margin-left: 120px}
/* Remove margins from "page content" on small screens */
@media only screen and (max-width: 600px) {#main {margin-left: 0}}
</style>
<body class="w3-black">

<!-- Icon Bar (Sidebar - hidden on small screens) -->
<nav class="w3-sidebar w3-bar-block w3-small w3-hide-small w3-center">
  <!-- Avatar image in top left corner -->
  <a href="#" class="w3-bar-item w3-button w3-padding-large w3-black">
    <i class="fa fa-home w3-xxlarge"></i>
    <p>HOME</p>
  </a>
</nav>

<!-- Navbar on small screens (Hidden on medium and large screens) -->
<div class="w3-top w3-hide-large w3-hide-medium" id="myNavbar">
  <div class="w3-bar w3-black w3-opacity w3-hover-opacity-off w3-center w3-small">
    <a href="#" class="w3-bar-item w3-button" style="width:25% !important">HOME</a>
    <a href="#about" class="w3-bar-item w3-button" style="width:25% !important">ABOUT</a>
  </div>
</div>

<!-- Page Content -->
<div class="w3-padding-large" id="main">
  <!-- Header/Home -->
  <header class="w3-container w3-padding-32 w3-center w3-black" id="home">
    <h1 class="w3-jumbo">There are no Recent Disaster-Related Tweets in """ + query + """<span class="w3-hide-small"></span></h1>
    <img src="https://ak.picdn.net/shutterstock/videos/29840074/thumb/1.jpg" alt="boy" class="w3-image" width="600" height="600">
  </header>

<!-- Contact Section -->
  <div class="w3-padding-64 w3-content w3-text-grey" id="contact">
    <p>Submit Phone Number:</p>

    <form action="/action_page.php" target="_blank">
      <p><input class="w3-input w3-padding-16" type="text" placeholder="Phone Number" required name="Number"></p>
      <p>
        <button class="w3-button w3-light-grey w3-padding-large" type="submit">
          <i class="fa fa-paper-plane"></i> SUBMIT NUMBER
        </button>
      </p>
    </form>
  <!-- End Contact Section -->
  </div>
   
    <!-- Footer -->
  <footer class="w3-content w3-padding-64 w3-text-grey w3-xlarge">
    <p class="w3-medium">Powered by <a href="https://www.w3schools.com/w3css/default.asp" target="_blank" class="w3-hover-text-green">w3.css</a></p>
  <!-- End footer -->
  </footer>

<!-- END PAGE CONTENT -->
</div>

</body>
</html>"""

In [131]:
global latest_tweet
latest_tweet = ""

In [132]:
def get_tweets(city, state):
  global latest_tweet
  tweets = api.search(q=city, tweet_mode='extended')
  txt = []
  for tweet in tweets:
    temp = tweet.full_text
    txt.append(temp)

  p, filtered = pred(txt)
  count = 0
  
  output = []
  count = 0



  for tweet in output:
    filtered.append(tweet)

  for i in range(0, len(tweets)):
    if p[i]:
      tweet = tweets[i]
      count += 1
      output.append(["@" + str(tweet.user.screen_name), str(tweet.created_at.strftime("%d/%m/%Y %H:%M:%S")), str(tweet.full_text)])



  #output = [
  #          ["@catlover99", "1/6/2021 14:29:10", "The Woodfield Sears entrance is completely destroyed."],
  #          ["@mirfontane", "1/6/2021 14:29:19", "Y'all see that car breaking through Sears at Schaumburg's Woodfield?"],
  #          ["@cactusjack", "1/6/2021 14:29:19", "We're not ever going back to Woodfield this year LOL"],
  #          ["@OMGronny", "1/6/2021 14:30:36", "OMG We have an active shooter at Woodfield Schaumburg!"],
  #          ["@cactusjack", "1/6/2021 14:32:33", "I'm hiding from this dude driving around in a car destroying everything someone help!"],
  #]
  #count = 2

  #filtered = [item[2] for item in output]





  
  if len(output) > 1:
    send_sms(from_="+12242987445", to='+12242399933', body="Latest disaster-related tweet in " + city + ": " + output[0][2])
  elif len(output) == 1:
    send_sms(from_="+12242987445", to='+12242399933', body="Latest disaster-related tweet in " + city + ": " + output[2])


  if count >= 1:
    return get_page(output, city, state, filtered)
  else:
    return get_blank(city, state)
  

In [133]:
import requests
import json
import re

def get_loc(ip):
  url = 'http://ipinfo.io/' + str(ip) + '/json'
  response = requests.get(url)
  data = json.loads(response.text)

  IP=data['ip']
  org=data['org']
  city = data['city']
  country=data['country']
  region=data['region']

  return city, region

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask
from flask import request

app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
    city, state = get_loc(request.headers.get('X-Forwarded-For'))
    return get_tweets(city, state)
    
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://ab0a9360bcd1.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


100%|██████████| 113/113 [00:00<00:00, 23158.23it/s]
127.0.0.1 - - [06/Jan/2021 19:21:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2021 19:21:00] "GET /favicon.ico HTTP/1.1" 404 -
